In [1]:
# Install google earth engine python API
# https://developers.google.com/earth-engine/python_install_manual
%pip install earthengine-api --upgrade pandas geopandas > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [83]:
# Get a bounds of the Sviatohirsk city from OpenStreetMap API
# https://wiki.openstreetmap.org/wiki/Overpass_API
import requests


response = requests.get('https://overpass-api.de/api/interpreter?data=[out:json];%20area[name=%22%D0%A1%D0%B2%D1%8F%D1%82%D0%BE%D0%B3%D1%96%D1%80%D1%81%D1%8C%D0%BA%22]%20-%3E%20.a;%20(%20relation(area.a)[admin_level];%20);%20out%20geom;')
data = response.json()
bounds = data['elements'][0]['bounds']

# Make a geojson file with bounds as a polygon
geoJSON = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry":
            {
                "type": "Polygon",
                "coordinates": [
                    [
                        [bounds['minlon'], bounds['minlat']],
                        [bounds['minlon'], bounds['maxlat']],
                        [bounds['maxlon'], bounds['maxlat']],
                        [bounds['maxlon'], bounds['minlat']],
                        [bounds['minlon'], bounds['minlat']]
                    ]
                ]
            }
        }
    ]
}

bounds

{'minlat': 48.871914,
 'minlon': 37.294625,
 'maxlat': 49.0618144,
 'maxlon': 37.6970164}

In [94]:
# Plot the bounds on the map with folium
# https://python-visualization.github.io/folium/

import folium
import json

m = folium.Map(location=[48.9, 37.5], zoom_start=11)
folium.GeoJson(json.dumps(geoJSON)).add_to(m)
m

In [7]:

import pandas as pd
import geopandas as gpd


In [62]:
# Translate Lat/Lon bounds of the city to Quadkeys to download building footprints
import math

def lat_lon_to_pixel(latitude, longitude, zoom):
    sinLatitude = math.sin(latitude * math.pi/180)
    pixelX = ((longitude + 180) / 360) * 256 * 2**zoom
    pixelY = (0.5 - math.log((1 + sinLatitude) / (1 - sinLatitude)) / (4 * math.pi)) * 256 * 2**zoom
    return pixelX, pixelY

def pixel_to_tile(pixelX, pixelY):
    return int(math.floor(pixelX / 256)), int(math.floor(pixelY / 256))

def tile_to_quadkey(tileX, tileY, zoom):
    quadkey = ""
    for i in range(zoom, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (tileX & mask) != 0:
            digit += 1
        if (tileY & mask) != 0:
            digit += 2
        quadkey += str(digit)
    return quadkey

# Define the bounding box for Sviatohirsk
minlat, minlon, maxlat, maxlon = bounds['minlat'], bounds['minlon'], bounds['maxlat'], bounds['maxlon']

# Choose a zoom level (adjust as needed to cover the city area)
zoom_level = 12

# Convert the bounding box corners to pixel coordinates
min_pixelX, min_pixelY = lat_lon_to_pixel(minlat, minlon, zoom_level)
max_pixelX, max_pixelY = lat_lon_to_pixel(maxlat, maxlon, zoom_level)

# Convert pixel coordinates to tile coordinates
min_tileX, min_tileY = pixel_to_tile(min_pixelX, min_pixelY)
max_tileX, max_tileY = pixel_to_tile(max_pixelX, max_pixelY)

# Find Quadkeys for each corner (you might need to find all Quadkeys in the range for full coverage)
quadkey_min = tile_to_quadkey(min_tileX, min_tileY, zoom_level)
quadkey_max = tile_to_quadkey(max_tileX, max_tileY, zoom_level)


# Remove three last digits from the quadkey
# quadkey_min = quadkey_min[:-3]
# quadkey_max = quadkey_max[:-3]

print('Quadkey coordinates of the city:')
quadkey_min, quadkey_max

Quadkey coordinates of the city:


('120330101000', '120312323302')

In [74]:
# Adjusting the approach to account for the full-length Quadkeys

# Define the full-length Quadkeys
def quadkey_to_tile(quadkey):
    tileX, tileY = 0, 0
    zoom = len(quadkey)
    for i in range(zoom, 0, -1):
        mask = 1 << (i - 1)
        if quadkey[zoom - i] == '1':
            tileX |= mask
        elif quadkey[zoom - i] == '2':
            tileY |= mask
        elif quadkey[zoom - i] == '3':
            tileX |= mask
            tileY |= mask
    return tileX, tileY, zoom

# Convert full-length Quadkeys back to tile coordinates
min_tileX, min_tileY, zoom = quadkey_to_tile(quadkey_min)
max_tileX, max_tileY, _ = quadkey_to_tile(quadkey_max)

# Ensure the min and max tile ranges are valid
min_tileX, max_tileX = sorted([min_tileX, max_tileX])
min_tileY, max_tileY = sorted([min_tileY, max_tileY])

# Generate list of all Quadkeys in the bounding box
all_quadkeys = []
for tileX in range(min_tileX, max_tileX + 1):
    for tileY in range(min_tileY, max_tileY + 1):
        all_quadkeys.append(tile_to_quadkey(tileX, tileY, len(quadkey_min)))

all_quadkeys_cropped = list(set(map(lambda x: int(x[:-3]), all_quadkeys)))
all_quadkeys_cropped

[120312323, 120330101]

In [78]:
# https://github.com/microsoft/GlobalMLBuildingFootprints/blob/main/scripts/make-gis-friendly.py

# Get polygons of building footprints for selected bounding box from Microsoft Building Footprints dataset
from shapely.geometry import shape
import os

# this is the name of the geography you want to retrieve. update to meet your needs
location = 'Ukraine'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
ukraine_links = dataset_links[dataset_links.Location == location]
sviatohirsk_links = ukraine_links[ukraine_links.QuadKey.isin(all_quadkeys_cropped)]

print('Total Links: ', len(sviatohirsk_links))

footprints = []
for _, row in sviatohirsk_links.iterrows():
    df = pd.read_json(row.Url, lines=True)
    df['geometry'] = df['geometry'].apply(shape)
    footprints.append(gpd.GeoDataFrame(df, crs=4326))

footprints = pd.concat(footprints)

Total Links:  2


In [79]:
# Filter footprints by the city bounds
footprints.cx[minlon:maxlon, minlat:maxlat].to_file('./footprints.geojson', driver='GeoJSON')